<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/opeanai_aws_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install colab-env --upgrade
!pip install openai
!pip install boto3
!pip install json

import colab_env
import os

aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
region=os.getenv("region")
output=os.getenv("output")
FunctionName_ARN=os.getenv("AWS_LAMBDA_ARN")

In [161]:
import json
import openai
import boto3

### chat completions
openai.api_key = get_api_key()

def lambda_handler_cc(prompt):
    stream = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")



### completions


def lambda_handler(prompt):
    #MODEL	DESCRIPTION	CONTEXT WINDOW	TRAINING DATA
#gpt-4-1106-preview
    model_to_use = "gpt-3.5-turbo-instruct"
    #model_to_use = "gpt-4-0314"
    #model_to_use = "gpt-4-1106-preview"
    #input_prompt="Write an email to Elon Musk asking him why he bought Twitter for such a huge amount"
    input_prompt=prompt

    openai.api_key = get_api_key()
    response = openai.completions.create(
      model=model_to_use,
      prompt=input_prompt,
      temperature=0.8,
      max_tokens=512,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )
    #print(response)
    return response
    #text_response = response['choices'][0]['text'].strip()


def get_api_key():
    lambda_client = boto3.client('lambda')
    response = lambda_client.invoke(
            FunctionName = FunctionName_ARN,
            InvocationType = 'RequestResponse'
        )

    openai_api_key = json.load(response['Payload'])['body']['api_key']
    return openai_api_key


In [177]:

prompt = "what is the 40% of 30?"
prompt = "what is the 20.5% of 40?"
prompt = "what is the 30% of 650?"
prompt = "As a data scientist, \ncan you explain the concept of regularization in machine learning?"
prompt="Write an email to Elon Musk asking him why he bought Twitter for such a huge amount"

prompt='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill.\n How many dollars did I get back? Explain first before answering.'
#prompt='Which country has the most natural lakes? Answer with only the country name.'
print('Prompt: %s'%prompt)
print()
print('chat completions - gpt4')
print()
lambda_handler_cc(prompt)

print()
print('===========')
print()
response=lambda_handler(prompt)
print()
#print('Answer:%s \r\n'%response.choices[0].text)

print('Answer (gpt 3.5):%s \n'%response.choices[0].text, sep='', flush=True)

Prompt: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill.
 How many dollars did I get back? Explain first before answering.

chat completions - gpt4

First, we need to calculate the total cost of the ice cream cones. With 6 kids and each cone costing $1.25, we multiply 6 by 1.25, getting a total of $7.5. 

Then, to find out how much change you got back, subtract the total cost of the ice cream cones from the $10 bill. This gives us 10-7.5, which equals $2.5.

So, you got back $2.5 as change.


Answer (gpt 3.5):

You would get $2.50 back. 

Explanation: Since each cone was $1.25, 6 cones would cost $7.50. When you pay with a $10 bill, you are essentially giving $10 to the cashier and receiving $10 worth of goods. Since the ice cream only cost $7.50, the cashier would give you the remaining $2.50 in change.  

